## SITCOM-710: Analysis of TMA velocity, acceleration, and jerk

This notebook is designed to analyze the velocity, acceleration, and jerk of TMA slews.

Current plots:
- The velocity, acceleration, and jerk over the time period of a single slew for both azimuth and elevation
- Histogram of maximums (velocity, acceleration, and jerk) of each of the slews within a choosen range of dayObs


In [ ]:
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import pandas as pd
from astropy.time import Time
from scipy.interpolate import UnivariateSpline
from scipy.interpolate import interp1d
from scipy.signal import find_peaks
from scipy.signal import savgol_filter

from lsst.summit.utils.tmaUtils import TMAEventMaker, getSlewsFromEventList
from lsst.summit.utils.efdUtils import getEfdData, calcNextDay
from lsst.sitcom import vandv

## Main Class

In [ ]:
class SlewData:
    """
    Queries, analyzes, and holds slew data for a range of dayObs
    
    Parameters
    
    Attributes
    """
    
    def __init__(self, dayStart, dayEnd, event_maker, spline_fit = "spline", padding = 0):
        self.day_range = self.get_day_range(dayStart, dayEnd)
        self.event_maker = event_maker
        self.spline_fit = spline_fit
        self.padding = padding
        self.all_data = self.get_spline_data()
        self.max_data = self.get_max_frame()
        
    def get_day_range(self, dayStart, dayEnd):
        if dayStart > dayEnd:
            assert False, "dayStart is after dayEnd"
        
        dayRange = []
        
        while dayStart <= dayEnd:
            dayRange.append(dayStart)
            dayStart = calcNextDay(dayStart)
        
        return dayRange

    def get_spline_frame(self, dayObs, index, fullAzTimestamp, fullElTimestamp, az_position, az_velocity, el_position, el_velocity):
        """
        Create a data frame for all original and fitted data
        """
        npoints = int(np.max([np.round((fullAzTimestamp[-1]-fullAzTimestamp[0])/0.01/1e3,0)*1e3, 4000])) # clarify what this is doing
        plotAzXs = np.linspace(fullAzTimestamp[0], fullAzTimestamp[-1], npoints)
        plotElXs = np.linspace(fullElTimestamp[0], fullElTimestamp[-1], npoints)

        kernel_size = len(fullAzTimestamp)
        kernel = np.ones(kernel_size)/kernel_size
        s = 0 # smoothing factor

        if self.spline_fit == "spline":
            # input: times, positions, velocities, interpPoints, kernel, smoothing factor
            azPosSpline, azVelSpline, azAccSpline, azJerkSpline = self.get_univariate_splines(fullAzTimestamp, az_position, az_velocity, plotAzXs, kernel, s) 
            elPosSpline, elVelSpline, elAccSpline, elJerkSpline = self.get_univariate_splines(fullElTimestamp, el_position, el_velocity, plotElXs, kernel, s)
        elif self.spline_fit == "savgol":
            azPosSpline, azVelSpline, azAccSpline, azJerkSpline = self.get_savgol_splines(fullAzTimestamp, az_position, plotAzXs)
            elPosSpline, elVelSpline, elAccSpline, elJerkSpline = self.get_savgol_splines(fullElTimestamp, el_position, plotElXs)
        else:
            assert False, self.spline_method + " is not a valid spline method. Use either \"spline\" or \"savgol\"."

        spline_frame=pd.DataFrame({
                    "slew_index":index,
                    "day":dayObs,
                    "azZeroTime":fullAzTimestamp.values[0],
                    "elZeroTime":fullElTimestamp.values[0],
                    "azTime":plotAzXs,
                    "azPosition":azPosSpline,
                    "azVelocity":azVelSpline,          
                    "azAcceleration":azAccSpline,
                    "azJerk":azJerkSpline,
                    "elTime":plotElXs,
                    "elPosition":elPosSpline,
                    "elVelocity":elVelSpline,          
                    "elAcceleration":elAccSpline,
                    "elJerk":elJerkSpline
        })

        return spline_frame

    def get_univariate_splines(self, times, positions, velocities, interpPoints, kernel, smoothingFactor):
        try:
            posSpline = UnivariateSpline(times, position, s=0)
        except:
            #if there are duplicate time measurements remove them  (this occured on 
            # 23/11/22-23 and 21-22)
            times, indexes=np.unique(times, return_index=True)
            positions=positions[indexes]
            velocities=velocities[indexes]

            posSpline = UnivariateSpline(times, positions, s=0)
            velSpline1  = UnivariateSpline(times, velocities, s=0) 

        # Now smooth the derivative before differentiating again
        smoothedVel = np.convolve(velSpline1(interpPoints), kernel, mode='same')
        velSpline = UnivariateSpline(interpPoints, smoothedVel, s=smoothingFactor)
        accSpline1 = velSpline.derivative(n=1)
        smoothedAcc = np.convolve(accSpline1(interpPoints), kernel, mode='same')
        # Now smooth the derivative before differentiating again
        accSpline = UnivariateSpline(interpPoints, smoothedAcc, s=smoothingFactor)
        jerkSpline = accSpline.derivative(n=1)
        
        return posSpline(interpPoints), velSpline(interpPoints), accSpline(interpPoints), jerkSpline(interpPoints)
    
    def savgolFilter(self, times, positions,interpPoints, window=200, deriv=1, smoothingFactor = 0.01): 
        positionSpline = UnivariateSpline(times, positions, s=smoothingFactor)(interpPoints) 
        derivativePoints = savgol_filter(positionSpline, window_length=window, mode="mirror",
                                        deriv=deriv, polyorder=3, delta=(interpPoints[1]-interpPoints[0])) 
        return derivativePoints

    def get_savgol_splines(self, times, positions, interpPoints):
        posSpline = UnivariateSpline(times, positions, s=0)(interpPoints)
        velSpline = self.savgolFilter(times, positions, interpPoints, window=200, deriv=1, smoothingFactor=0.01)
        accSpline = self.savgolFilter(times, positions, interpPoints, window=200, deriv=2, smoothingFactor=0.01)
        jerkSpline = self.savgolFilter(times, positions, interpPoints, window=200, deriv=3, smoothingFactor=0.01)
        return posSpline, velSpline, accSpline, jerkSpline
    
    def get_spline_data(self):
        topic_az = "lsst.sal.MTMount.azimuth"
        topic_el = "lsst.sal.MTMount.elevation"
        topic_columns = ["actualPosition", "actualVelocity", "timestamp"]

        spline_frame = pd.DataFrame()

        for day in self.day_range:
            slew_events = getSlewsFromEventList(self.event_maker.getEvents(day))
            print(f'Found {len(slew_events)} slews for {day=}')

            if len(slew_events) == 0:
                continue

            for slew in range(len(slew_events)):
                data_az = getEfdData(
                    self.event_maker.client, 
                    topic_az, columns = topic_columns,
                    prePadding = self.padding,
                    postPadding = self.padding,
                    event=slew_events[slew]
                )
                data_el = getEfdData(
                    self.event_maker.client, 
                    topic_el, columns = topic_columns,
                    prePadding = self.padding,
                    postPadding = self.padding,
                    event=slew_events[slew]
                )
                
                # check if the event has enough data to fit a spline
                if (data_az.shape[0] < 4) or (data_el.shape[0] < 4):
                    continue

                spline_frame_single = self.get_spline_frame(
                    day,
                    slew,
                    data_az["timestamp"],
                    data_el["timestamp"], 
                    data_az["actualPosition"], 
                    data_az["actualVelocity"], 
                    data_el["actualPosition"], 
                    data_el["actualVelocity"]
                )
                
                spline_frame = pd.concat([spline_frame, spline_frame_single], ignore_index=True)
        
        return spline_frame
    
    def get_max_frame(self):
        slew_num = []
        day_num = []
        slew_time = [] # final time

        az_vel_max = []
        el_vel_max = []

        az_acc_max = []
        el_acc_max = []

        az_jerk_max = []
        el_jerk_max = []

        for i in np.unique(self.all_data['day']):
            slew_day = (self.all_data['day']==i)
            for j in np.unique(self.all_data['slew_index'][slew_day]):
                slew_id = slew_day & (self.all_data['slew_index']==j)

                az_vel_max.append(abs(self.all_data.loc[slew_id,"azVelocity"]).max())
                el_vel_max.append(abs(self.all_data.loc[slew_id,"elVelocity"]).max())

                az_acc_max.append(abs(self.all_data.loc[slew_id,"azAcceleration"]).max())
                el_acc_max.append(abs(self.all_data.loc[slew_id,"elAcceleration"]).max())

                az_jerk_max.append(abs(self.all_data.loc[slew_id,"azJerk"]).max())
                el_jerk_max.append(abs(self.all_data.loc[slew_id,"elJerk"]).max())

                slew_num.append(j)
                day_num.append(i)

                slew_time.append(np.max([self.all_data.loc[slew_id,"azTime"].max(),self.all_data.loc[slew_id,"elTime"].max()]))

        max_frame=pd.DataFrame({
            "day":day_num,
            "slew":slew_num,
            "az_vel":az_vel_max,
            "az_acc":az_acc_max,
            "az_jerk":az_jerk_max,
            "el_vel":el_vel_max,
            "el_acc":el_acc_max,
            "el_jerk":el_jerk_max
        })

        return max_frame

### Functions for plotting

In [ ]:
# generate histograms for max values during slews
def plot_max_hist(max_frame, limitsBool, logBool, fit, padding):
    padding = str(padding)
    num_slews= str(max_frame.shape[0])
    design_color = "green"
    max_color = "orange"
    
    first_day = max_frame['day'].min()
    last_day = max_frame['day'].max()

    fig,axs = plt.subplots(3, 2, dpi=175, figsize=(10,5), sharex=False)
    plt.subplots_adjust(wspace=0.3, hspace=0.5)
    if len(np.unique(max_frame['day'])) > 1:
        if logBool:
            plt.suptitle(f"Maximums for {first_day} - {last_day} -- Slews: " + num_slews + "\nLog Count -- Fit: " + fit + " -- Padding: " + padding, fontsize = 11)
        else:
            plt.suptitle(f"Maximums for {first_day} - {last_day} -- Slews: " + num_slews + "\n Fit: " + fit + " -- Padding: " + padding, fontsize = 14)
    else:
        if logBool:
            plt.suptitle(f"Maximums for {first_day} -- Slews: " + num_slews + "\nLog Count -- Fit: " + fit + " -- Padding: " + padding, fontsize = 11)
        else:
            plt.suptitle(f"Maximums for {first_day} -- Slews: " + num_slews + "\nFit: " + fit + " -- Padding: " + padding, fontsize = 14)

    # bins for each type. The middle parameter is based on the max spec limit
    velbins = np.linspace(0, az_limit_dict["max_velocity"], 100) 
    accbins = np.linspace(0, az_limit_dict["max_acceleration"], 100)
    jerkbins = np.linspace(0, az_limit_dict["max_jerk"], 100)
    
    plt.subplot(3,2,1)
    plt.hist(max_frame["az_vel"], log=logBool, color="tab:blue", bins=velbins)
    if limitsBool:
        # only require counts output from plt.hist
        counts, bins, patches = plt.hist(max_frame["az_vel"], color="tab:blue", bins=velbins)
        plotHistAzDesignLim("design_velocity", "max_velocity", 0, np.max(counts), design_color, max_color)
    plt.title(f"Azimuth")
    plt.ylabel("Velcoity Count")
    plt.xlabel("deg/s")

    plt.subplot(3,2,2)
    plt.hist(max_frame["el_vel"], log=logBool, color="tab:blue", bins=velbins)
    if limitsBool == True:
        counts, bins, patches = plt.hist(max_frame["el_vel"], color="tab:blue", bins=velbins)
        plotHistElDesignLim("design_velocity", "max_velocity", 0, np.max(counts), design_color, max_color)
    plt.title(f"Elevation")
    plt.xlabel("deg/s")

    plt.subplot(3,2,3)
    plt.hist(max_frame["az_acc"], log=logBool, color="tab:blue", bins=accbins)
    if limitsBool == True:
        counts, bins, patches = plt.hist(max_frame["az_acc"], color="tab:blue", bins=accbins)
        plotHistAzDesignLim("design_acceleration", "max_acceleration", 0, np.max(counts), design_color, max_color)
    plt.ylabel("Acceleration Count")
    plt.xlabel("deg/s^2")

    plt.subplot(3,2,4)
    plt.hist(max_frame["el_acc"], log=logBool, color="tab:blue", bins=accbins)
    if limitsBool == True:
        counts, bins, patches = plt.hist(max_frame["el_acc"], color="tab:blue", bins=accbins)
        plotHistElDesignLim("design_acceleration", "max_acceleration", 0, np.max(counts), design_color, max_color)
    plt.xlabel("deg/s^2")

    plt.subplot(3,2,5)
    plt.hist(max_frame["az_jerk"], log=logBool, color="tab:blue", bins=jerkbins)
    if limitsBool == True:
        counts, bins, patches = plt.hist(max_frame["az_jerk"], color="tab:blue", bins=jerkbins)
        plotHistAzDesignLim("design_jerk", "max_jerk", 0, np.max(counts), design_color, max_color)
    plt.ylabel("Jerk Count")
    plt.xlabel("deg/s^3")

    plt.subplot(3,2,6)
    plt.hist(max_frame["el_jerk"], log=logBool, color="tab:blue", bins=jerkbins)
    if limitsBool == True:
        counts, bins, patches = plt.hist(max_frame["el_jerk"], color="tab:blue", bins=jerkbins)
        plotHistElDesignLim("design_jerk", "max_jerk", 0, np.max(counts), design_color, max_color)
    plt.xlabel("deg/s^3")

    plt.show()

In [ ]:
# define functions to add limits to slew profile for neatness
def plotAzDesignlim(design_input, max_input, xmin, xmax, design_color, max_color):
    plt.hlines(az_limit_dict[design_input], xmin=xmin, xmax=xmax, color = design_color)
    plt.hlines(-az_limit_dict[design_input], xmin=xmin, xmax=xmax, color = design_color)
    plt.hlines(az_limit_dict[max_input], xmin=xmin, xmax=xmax, color = max_color)
    plt.hlines(-az_limit_dict[max_input], xmin=xmin, xmax=xmax, color = max_color)
    
def plotElDesignlim(design_input, max_input, xmin, xmax, design_color, max_color):
    plt.hlines(el_limit_dict[design_input], xmin=xmin, xmax=xmax, color = design_color)
    plt.hlines(-el_limit_dict[design_input], xmin=xmin, xmax=xmax, color = design_color)
    plt.hlines(el_limit_dict[max_input], xmin=xmin, xmax=xmax, color = max_color)
    plt.hlines(-el_limit_dict[max_input], xmin=xmin, xmax=xmax, color = max_color)

def plotHistAzDesignLim(design_input, max_input, ymin, ymax, design_color, max_color):
    plt.vlines(az_limit_dict[design_input], ymin=ymin, ymax=ymax, color = design_color)
    plt.vlines(az_limit_dict[max_input], ymin=ymin, ymax=ymax, color = max_color)
    
def plotHistElDesignLim(design_input, max_input, ymin, ymax, design_color, max_color):
    plt.vlines(el_limit_dict[design_input], ymin=ymin, ymax=ymax, color = design_color)
    plt.vlines(el_limit_dict[max_input], ymin=ymin, ymax=ymax, color = max_color)

In [ ]:
# Generate plots for a slew profile in both azimuth and elevation
# for position, velocity, acceleration, and jerk
# limitsBool when set to true adds spec limits to graphs
# TO-DO: add bool to offer choice between showing plot or saving plot to a file
# TO-DO: add legend to indicate fit data vs real data

def slew_profile_plot(spline_frame, dayObs, slew_index, limitsBool):
    # create a spline frame for a single slew
    slew_frame = spline_frame.loc[((spline_frame['day']==dayObs) & (spline_frame['slew_index']==slew_index))]
    
    if len(slew_frame) == 0:
        assert False, f"There is no data for slew {slew_index} of dayObs {dayObs}"
    
    # format time used in the title
    title_time = Time(slew_frame['azZeroTime'].iloc[[0]], format = 'unix').iso
    
    # get the relative times for the x-axis
    azRelativeTimes = slew_frame['azTime'] - slew_frame['azZeroTime']
    elRelativeTimes = slew_frame['elTime'] - slew_frame['elZeroTime']

    fig,axs = plt.subplots(4, 2, dpi=175, figsize=(10,5), sharex=True)
    plt.subplots_adjust(wspace=0.3, hspace=0.5)
    plt.suptitle(f"TMA Slew Number {slew_index} \n Time: {title_time}", fontsize = 12, y = 1.00)

    # make it easier to change variables across subplots
    mark = "x"
    mark_color = "purple"
    mark_size = 30
    line_width = 2
    az_color = "red"
    el_color = "blue"
    design_color = "green"
    max_color = "orange"
    opacity = 0.5
    
    plt.subplot(4,2,1)
    plt.plot(azRelativeTimes, slew_frame['azPosition'], lw=line_width, color=az_color, label='Spline fit')
    plt.scatter(azRelativeTimes, slew_frame['azPosition'], marker=mark, color=mark_color,alpha=opacity, s=mark_size, label='Measured points')
    plt.title(f"Azimuth")
    plt.ylabel("Degrees")

    plt.subplot(4,2,2)
    plt.plot(elRelativeTimes, slew_frame['elPosition'], lw=line_width, color=el_color, label='Spline fit')
    plt.scatter(elRelativeTimes, slew_frame['elPosition'], marker=mark, color=mark_color,alpha=opacity, s=mark_size, label='Measured points')
    plt.title(f"Elevation")

    plt.subplot(4,2,3)
    plt.plot(azRelativeTimes, slew_frame['azVelocity'], lw=line_width, color=az_color, label='Spline fit')
    plt.scatter(azRelativeTimes, slew_frame['azVelocity'], marker=mark, color=mark_color,alpha=opacity, s=mark_size, label='Measured points')
    if limitsBool == True:
        plotAzDesignlim("design_velocity", "max_velocity", azRelativeTimes.iloc[[0]], azRelativeTimes.iloc[[-1]], design_color, max_color)
    plt.ylabel("Deg/sec")

    plt.subplot(4,2,4)
    plt.plot(elRelativeTimes, slew_frame['elVelocity'], lw=line_width, color=el_color, label='Spline fit')
    plt.scatter(elRelativeTimes, slew_frame['elVelocity'], marker=mark, color=mark_color,alpha=opacity, s=mark_size, label='Measured points')
    if limitsBool == True:
        plotElDesignlim("design_velocity", "max_velocity", elRelativeTimes.iloc[[0]], elRelativeTimes.iloc[[-1]], design_color, max_color)

    plt.subplot(4,2,5)
    plt.plot(azRelativeTimes, slew_frame['azAcceleration'], lw=line_width, color=az_color, label='Spline fit')
    if limitsBool == True:
        plotAzDesignlim("design_acceleration", "max_acceleration", azRelativeTimes.iloc[[0]], azRelativeTimes.iloc[[-1]], design_color, max_color)
    plt.ylabel("Deg/sec^2")

    plt.subplot(4,2,6)
    plt.plot(elRelativeTimes, slew_frame['elAcceleration'], lw=line_width, color=el_color, label='Spline fit')
    if limitsBool == True:
        plotElDesignlim("design_acceleration", "max_acceleration", elRelativeTimes.iloc[[0]], elRelativeTimes.iloc[[-1]], design_color, max_color)

    plt.subplot(4,2,7)
    plt.plot(azRelativeTimes, slew_frame['azJerk'], lw=line_width, color=az_color, label='Spline fit')
    if limitsBool == True:
        plotAzDesignlim("design_jerk", "max_jerk", azRelativeTimes.iloc[[0]], azRelativeTimes.iloc[[-1]], design_color, max_color)
    plt.ylabel("Deg/sec^3")
    plt.xlabel("seconds")

    plt.subplot(4,2,8)
    plt.plot(elRelativeTimes, slew_frame['elJerk'], lw=line_width, color=el_color, label='Spline fit')
    if limitsBool == True:
        plotElDesignlim("design_jerk", "max_jerk", elRelativeTimes.iloc[[0]], elRelativeTimes.iloc[[-1]], design_color, max_color)
    plt.xlabel("seconds")

    plt.show()

## Defining parameters

In [ ]:
# define limits from science requirements document (LTS-103 2.2.2) for plotting
# units in deg/s - deg/s^2 - deg/s^3
el_limit_dict={
    "max_velocity": 5.25,
    "max_acceleration": 5.25,
    "max_jerk": 21,
    "design_velocity": 3.5,
    "design_acceleration": 3.5,
    "design_jerk": 14,
}
az_limit_dict={
    "max_velocity": 10.5,
    "max_acceleration": 10.5,
    "max_jerk": 42,
    "design_velocity": 7,
    "design_acceleration": 7,
    "design_jerk": 28,
}

In [ ]:
# spline fit type, can be either "spline" for univariate (the default) or "savgol"
spline_filter = "spline"

In [ ]:
padding = 0

## Plots

Store all queried data in a single variable from the days defined above

In [ ]:
event_maker = TMAEventMaker()
data = SlewData(20230501, 20230831, event_maker, spline_filter, padding=padding)

Identify the maximums of the velocity, acceleration, and jerk of each slew and plot statistics

In [ ]:
%matplotlib inline

In [ ]:
print("Final number of slews: " + str(data.max_data.shape[0]))

In [ ]:
plot_max_hist(data.max_data, True, True, spline_filter, padding)

Pick a slew to show a single slew motion analysis plot

In [ ]:
slew_profile_plot(data.all_data, 20230731, 3, False)